In [49]:
import numpy as np
import pandas as pd
#from sklearn.feature_extraction.text import CountVectorizer


#from sklearn.model_selection import train_test_split
#from sklearn.datasets import load_iris

In [50]:
# iris = load_iris()
# X = iris.data
# y = iris.target

In [51]:
# X_train, X_test, y_train, y_test = train_test_split(X,y)

In [52]:
# iris.keys()

In [53]:
#drug_train = pd.read_csv("Data/drugLibTrain_raw.tsv",delimiter='\t',encoding='utf-8')
#drug_test = pd.read_csv("Data/drugLibTest_raw.tsv",delimiter='\t',encoding='utf-8')

In [54]:
# drug=pd.concat([drug_train,drug_test], axis=0, join='outer', ignore_index=False, keys=None,
#           levels=None, names=None, verify_integrity=False, copy=True)

In [55]:
# print(drug_train.shape)
# print(drug_test.shape)
# print(drug.shape)

In [56]:
#drug['rating'].nunique()

In [57]:
#drug.head()

In [58]:
#drug.columns

In [59]:
#drug['sideEffects'].unique(),drug['sideEffects'].nunique()

# Gaussian Discriminant Analysis(GDA)

In [60]:
class GaussianDiscriminantAnalysis:
    def __init__(self,epsilon=10e-10):
        self.epsilon=epsilon
        
    def ph(self,y):
        phi=[]
        for k in self.classes:#range(self.nb_class):
            phi.append((np.sum(y==k))/len(y))
        return phi   
    
    def phi(self, y):
        return (np.sum(y==1))/len(y)
    
    def mu_computed(self,X,y):
        list_of_mu=[]
        for k in self.classes: #range(self.nb_class):
            list_of_mu.append((np.sum(X[y==k],axis=0))/np.sum(y==k))
            
        return list_of_mu
    
    def covariance(self,X,y):
        sigma= np.zeros((len(X),len(X)))
        M = np.zeros_like(X)
        for i in range(1,self.nb_class):
            M[y== i]=self.list_mu[i]
        return (((X-M).T)@(X-M))/len(y)
    
    def prob_class(self,X):
        half_len = X.shape[1]/2
        det_cvar = np.sqrt(np.linalg.det(self.cvar ))
        inv_cvar = np.linalg.inv(self.cvar + (self.epsilon*np.eye(len(self.cvar))))
        list_prob=[]
        for i in self.classes:#range(self.nb_class):
            A=((X - self.list_mu[i])@inv_cvar)
            #print('dima',A)
            B=(X-self.list_mu[i])
            #print('dimb',B)
            C=np.sum(np.multiply(A,B),axis=1)
            list_prob.append(np.exp( -0.5*C)*(1/((2*np.pi)**half_len)*det_cvar))
        return list_prob
    
    def train(self, X, y):
        self.nb_class = len(np.unique(y))
        self.classes = np.unique(y)
        self.fi = self.ph(y)
        self.list_mu = self.mu_computed(X,y)
        self.cvar = self.covariance(X, y)
        
    def predict(self, X):        
        proby=self.fi
        list_probclass= self.prob_class(X)
        predict_prob=[]
        for i in self.classes:#range(self.nb_class):
            
            predict_prob.append(list_probclass[i]*proby[i])
        
   
        return np.argmax(np.array(predict_prob).T,axis=1) 

In [61]:
#GDA = GaussianDiscriminantAnalysis()

In [62]:
#GDA.train(X_train,y_train)

In [63]:
#X_test.shape

In [64]:
#prediction=GDA.predict(X_test)

In [65]:
#np.sum(prediction == y_test) / len(y_test)*100

#  Naive Bayes (NB)

In [66]:
# text = drug[['commentsReview','benefitsReview','sideEffectsReview']].apply(lambda x: 
#                                                     ' '.join(str(s) for s in x), axis=1)
# c=CountVectorizer(max_features=5,binary=True)
# j = c.fit_transform(text)
# X = j.toarray()
# y=drug['rating']
# X_train, X_test, y_train, y_test = train_test_split(X,y)

In [67]:
class BernoulliNaiveBayes:
    
    def __init__(self):
        pass
        
        
    def phiy(self,y): 
        phi=[]
        for k in self.classes:
            phi.append(np.sum(y==k)/len(y))
        return phi
    
    
    def phix1y(self,X,y):
        phi_list=[]
        for k in self.classes:
            phix=[]
            for i in range(X.shape[1]):
                X_i=X[:,i]
                select_y=X_i[y==k]
                phix.append((np.sum(select_y==1)+1)/(np.sum(y==k)+X.shape[1]))
            phi_list.append(phix)
        return phi_list
    
    def phix0y(self,X,y):
        phi_list=[]
        for k in self.classes:
            phix=[]
            for i in range(X.shape[1]):
                X_i=X[:,i]
                select_y=X_i[y==k]
                phix.append((np.sum(select_y==0)+1)/(np.sum(y==k)+X.shape[1]))
            phi_list.append(phix)
        return phi_list
    
    def probabxy(self,X):
        
        probxyk=[]
        arr1=np.zeros_like(X)
        
        for k in self.classes:
            probxy1=[]
            for i in range(X.shape[0]):
                X_i=X[i,:]
                print('obsevation',X[i,:].shape)
                prob=1
                for j in range(len(X_i)):

                    if X_i[j]==1:
                        print('in if',self.fix1y[k])
                        prob *=  self.fix1y[k-1][j]

                    else:
                        print('in else',self.fix0y[k])
                        prob *= self.fix0y[k-1][j]
                    arr1[i,j]=prob
                probxy1.append(prob)
            #print(np.array(probxy1).shape)
            print('out ',k)
            probxyk.append(probxy1)
        return  probxyk 

    def fit_train(self, X,y):
        self.nb_class = len(np.unique(y))
        self.classes = np.unique(y)
        self.fiy = self.phiy(y)
        self.fix1y = np.array(self.phix1y(X,y))
        self.fix0y = np.array(self.phix0y(X,y))
        #print(self.fix0y.shape)
        #print(self.fix0y[0:10])
        
    def predict(self,X) :
        proby=self.fiy
        list_probclass= self.probabxy(X)
        
        predict_prob=[]
        predict_prob=np.array(list_probclass).T*proby
        
        return  np.argmax(predict_prob,axis=1)  

In [68]:
#NB = BernoulliNaiveBayes()

In [69]:
#NB.fit_train(X_train,y_train)

In [70]:
#prediction=NB.predict(X_test)

In [71]:
#prediction

In [72]:
#y_test

In [73]:
#np.sum(prediction==y_test)/len(y_test)*100